In [1]:
import pickle
import requests
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateutil.parser
from datetime import datetime
import locale
#from locale import atof
import patsy
#locale.setlocale(locale.LC_NUMERIC, '')
#'en_GB.UTF-8'

%matplotlib inline
from IPython.display import Image

open the data frame with data from box office mojo and imdb

In [2]:
with open("df_merge.pkl", 'rb') as picklefile: 
    df = pickle.load(picklefile)

Convert relevant columns to usuable format for model. ie- if value of column is date, make datetime object. 

In [3]:
#remove "min" from runtime and convert to integer
df['Runtime'] = df['Runtime'].str.rstrip(' min')
df['Runtime'] = df['Runtime'].apply(str)


In [4]:
#remove "runtime" (in hours) from data frame- there are two, one in minutes, one in hours
df = df.drop(['runtime'], 1)

In [5]:
#remove any movies that aren't comedy
df = df[df.genre.str.contains("Comedy","comedy") == True]

In [6]:
df['Released'] = df['Released'].apply(str)
df["distributor"] = df["distributor"].apply(str)
df["genre"] = df["genre"].apply(str)
df["rating"] = df["rating"]

#df['Released']
#pd.to_datetime(df['Released'], format='%d%b%Y')

In [9]:
#create datetime variable
dt = datetime.strptime("October 1, 1999",'%B %d, %Y')

In [10]:
df.loc[df["release_date"].str.contains("N/A"), "release_date"] = np.nan

In [11]:
#delete two movies that are messing up date function
df = df.drop([590,258,455,456])


In [12]:
#parse release_dates
df["release_date"] = df["release_date"].apply(str)
df["release_datetime"] = df["release_date"].apply(lambda x: datetime.strptime(x, '%B %d, %Y'))


In [13]:
#convert release_date to datetime object
df['release_date'] = pd.to_datetime(df['release_datetime'])

Remove columns that cannot be used for regression

In [45]:
#find N/A values
#df[df.tomatoMeter.str.contains("N/A")]

Add NaN values for missing N/A data

In [15]:
df.loc[df["tomatoMeter"].str.contains("N/A"), "tomatoMeter"] = np.nan

In [16]:
df.loc[df["tomatoRating"].str.contains("N/A"), "tomatoRating"] = np.nan

In [17]:
df.loc[df["tomatoReviews"].str.contains("N/A"), "tomatoReviews"] = np.nan

In [18]:
df.loc[df["tomatoRotten"].str.contains("N/A"), "tomatoRotten"] = np.nan
df.loc[df["tomatoUserRating"].str.contains("N/A"), "tomatoUserRating"] = np.nan
df.loc[df["tomatoUserReviews"].str.contains("N/A"), "tomatoUserReviews"] = np.nan
df.loc[df["tomatoFresh"].str.contains("N/A"), "tomatoFresh"] = np.nan
df.loc[df["imdbVotes"].str.contains("N/A"), "imdbVotes"] = np.nan
df.loc[df["imdbRating"].str.contains("N/A"), "imdbRating"] = np.nan
df.loc[df["Runtime"].str.contains("N/A"), "Runtime"] = np.nan

Convert numerical data to floats

In [19]:
df["imdbVotes"] = df["imdbVotes"].str.replace(r'[$,]', '').astype('float')

In [20]:
df["tomatoMeter"] = df["tomatoMeter"].apply(float)
df["tomatoRating"] = df["tomatoRating"].apply(float)
df["tomatoReviews"] = df["tomatoReviews"].apply(float)
df["tomatoRotten"] = df["tomatoRotten"].apply(float)
df["tomatoUserRating"] = df["tomatoUserRating"].apply(float)
df["tomatoUserReviews"] = df["tomatoUserReviews"].apply(float)
df["tomatoFresh"] = df["tomatoFresh"].apply(float)
df["imdbVotes"] = df["imdbVotes"].apply(float)
df["imdbRating"] = df["imdbRating"].apply(float)
df["Runtime"] = df["Runtime"].apply(float)


Remove columns/features from df that cannot be used in regression

In [21]:
df = df.drop(['Plot','Poster',"tomatoConsensus","tomatoURL",
              "totalSeasons","imdbID","Type","Response","BoxOffice",
              "Website","Error","Genre","release_datetime","budget",
              "Year","Writer","Awards","distributor","tomatoImage",
              "Director","Actors","tomatoUserMeter", "Country","DVD",
              "Language","Metascore","Rated","Released"], 1)

create dummy variables for production and rating features

In [22]:

d_rating = patsy.dmatrix('rating', data=df,return_type='dataframe')
#y = df.join(d_rating)
d_production = patsy.dmatrix('Production', data=df,return_type='dataframe')
#z = df.join(d_production)

In [37]:
#df_join = d_rating.join(d_production)
df_production_rating = pd.concat([d_rating,d_production, df], axis=1)

In [39]:
#add dummy variables to main dataframe
df_rating = pd.concat([d_rating, df], axis=1)

In [40]:
#pickle dataframes for regression
"""with open('df_production_rating', 'wb') as picklefile:
    pickle.dump(df_production_rating , picklefile)"""

In [43]:
"""with open('df_rating', 'wb') as picklefile:
    pickle.dump(df_rating , picklefile)"""